<a href="https://colab.research.google.com/github/Savin97/tipigoProject2025/blob/main/Tipigo_Project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from datetime import timedelta

In [2]:
api_key = "S95EWM44XNYS7XZE"
symbol = "AAPL"

url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={api_key}"

response = requests.get(url)
data = response.json()
print(data)

{'symbol': 'AAPL', 'annualEarnings': [{'fiscalDateEnding': '2025-03-31', 'reportedEPS': '4.05'}, {'fiscalDateEnding': '2024-09-30', 'reportedEPS': '6.08'}, {'fiscalDateEnding': '2023-09-30', 'reportedEPS': '6.12'}, {'fiscalDateEnding': '2022-09-30', 'reportedEPS': '6.11'}, {'fiscalDateEnding': '2021-09-30', 'reportedEPS': '5.62'}, {'fiscalDateEnding': '2020-09-30', 'reportedEPS': '3.27'}, {'fiscalDateEnding': '2019-09-30', 'reportedEPS': '2.98'}, {'fiscalDateEnding': '2018-09-30', 'reportedEPS': '2.97'}, {'fiscalDateEnding': '2017-09-30', 'reportedEPS': '2.3'}, {'fiscalDateEnding': '2016-09-30', 'reportedEPS': '2.0675'}, {'fiscalDateEnding': '2015-09-30', 'reportedEPS': '2.3'}, {'fiscalDateEnding': '2014-09-30', 'reportedEPS': '1.6075'}, {'fiscalDateEnding': '2013-09-30', 'reportedEPS': '1.415'}, {'fiscalDateEnding': '2012-09-30', 'reportedEPS': '1.5775'}, {'fiscalDateEnding': '2011-09-30', 'reportedEPS': '0.9875'}, {'fiscalDateEnding': '2010-09-30', 'reportedEPS': '0.54'}, {'fiscalDat

In [3]:
url2 = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={api_key}"
response = requests.get(url2)
data_time_series = response.json()
print(data_time_series)

{'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}


In [ ]:
# --- Extract and clean the 8 most recent quarterly reports ---
quarterly = data['quarterlyEarnings'][:8]

df_eps = pd.DataFrame(quarterly)
df_eps = df_eps.rename(columns={
    'fiscalDateEnding': 'Quarter_End',
    'reportedDate': 'Earnings_Date',
    'reportedEPS': 'EPS_Actual',
    'estimatedEPS': 'EPS_Estimate',
    'surprise': 'EPS_Surprise',
    'surprisePercentage': 'Surprise_Pct'
})

# --- Convert types ---
df_eps['Quarter_End'] = pd.to_datetime(df_eps['Quarter_End'])
df_eps['Earnings_Date'] = pd.to_datetime(df_eps['Earnings_Date'])
df_eps['EPS_Actual'] = pd.to_numeric(df_eps['EPS_Actual'], errors='coerce')
df_eps['EPS_Estimate'] = pd.to_numeric(df_eps['EPS_Estimate'], errors='coerce')
df_eps['EPS_Surprise'] = pd.to_numeric(df_eps['EPS_Surprise'], errors='coerce')
df_eps['Surprise_Pct'] = pd.to_numeric(df_eps['Surprise_Pct'], errors='coerce') / 100  # convert to decimal

# --- Final clean DataFrame ---
df_eps = df_eps[['Earnings_Date', 'EPS_Actual', 'EPS_Estimate', 'EPS_Surprise', 'Surprise_Pct']]
print(df_eps)

  Earnings_Date  EPS_Actual  EPS_Estimate  EPS_Surprise  Surprise_Pct
0    2025-05-02        1.65          1.63          0.02      0.012270
1    2025-01-31        2.40          2.35          0.05      0.021277
2    2024-11-01        0.97          1.60         -0.63     -0.393750
3    2024-08-02        1.40          1.35          0.05      0.037037
4    2024-05-03        1.53          1.50          0.03      0.020000
5    2024-02-02        2.18          2.10          0.08      0.038095
6    2023-11-03        1.46          1.39          0.07      0.050360
7    2023-08-04        1.26          1.19          0.07      0.058824


In [ ]:
# --- Parse daily price time series ---
if "Time Series (Daily)" not in data_time_series:
    print("Error: Alpha Vantage did not return price data.")
else:
    ts = data_time_series["Time Series (Daily)"]
    df_price = pd.DataFrame.from_dict(ts, orient="index")
    df_price.index = pd.to_datetime(df_price.index)
    df_price = df_price.sort_index()
    df_price = df_price.rename(columns={"4. close": "Close"})
    df_price["Close"] = df_price["Close"].astype(float)

Error: Alpha Vantage did not return price data.


In [ ]:
results = []

for _, row in df_eps.iterrows():
    report_date = row['Earnings_Date']

    # Find actual trading day on/after earnings
    future = df_price[df_price.index >= report_date]
    if future.empty:
        continue
    actual_date = future.index[0]

    try:
        price_before = df_price.loc[actual_date]['Close']
        price_d1 = df_price.loc[actual_date + timedelta(days=1):].iloc[0]['Close']
        price_d5 = df_price.loc[actual_date + timedelta(days=1):].iloc[4]['Close']
        price_d10 = df_price.loc[actual_date + timedelta(days=1):].iloc[9]['Close']
    except IndexError:
        continue  # skip if not enough forward data

    results.append({
        "Earnings_Date": actual_date.date(),
        "EPS_Actual": row['EPS_Actual'],
        "EPS_Estimate": row['EPS_Estimate'],
        "EPS_Surprise": row['EPS_Surprise'],
        "Surprise_Pct": row['Surprise_Pct'],
        "Price_Before": price_before,
        "Price_D1": price_d1,
        "Price_D5": price_d5,
        "Price_D10": price_d10,
        "Return_D1": (price_d1 - price_before) / price_before,
        "Return_D5": (price_d5 - price_before) / price_before,
        "Return_D10": (price_d10 - price_before) / price_before
    })

final_df = pd.DataFrame(results)
print(final_df)


NameError: name 'df_price' is not defined